# Урок 9. Python применение продвинутых методов

**Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.**

In [2]:
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/A_B_tests/Результаты А_B.csv')

df.head(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,0,0
1,337,gate_30,38,1,0
2,377,gate_40,165,1,0
3,483,gate_40,1,0,0
4,488,gate_40,179,1,1
5,540,gate_40,187,1,1
6,1066,gate_30,0,0,0
7,1444,gate_40,2,0,0
8,1574,gate_40,108,1,1
9,1587,gate_40,153,1,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  int64 
 4   retention_7     90189 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.4+ MB


In [9]:
count_page = df.groupby('userid').agg({'version': 'nunique'})

In [11]:
count_page[count_page['version'] > 1].shape

(0, 1)

In [13]:
a_group = df[df['version']=='gate_30']
b_group = df[df['version']=='gate_40']
a_group, b_group

(        userid  version  sum_gamerounds  retention_1  retention_7
 0          116  gate_30               3            0            0
 1          337  gate_30              38            1            0
 6         1066  gate_30               0            0            0
 11        2101  gate_30               0            0            0
 13        2179  gate_30              39            1            0
 ...        ...      ...             ...          ...          ...
 90179  9998576  gate_30              14            1            0
 90180  9998623  gate_30               7            0            0
 90182  9999178  gate_30              21            1            0
 90183  9999349  gate_30              10            0            0
 90186  9999710  gate_30              28            1            0
 
 [44700 rows x 5 columns],
         userid  version  sum_gamerounds  retention_1  retention_7
 2          377  gate_40             165            1            0
 3          483  gate_40         

In [21]:
a_group.sum_gamerounds.mean(), a_group.retention_1.mean(), a_group.retention_7.mean()

(52.45626398210291, 0.4481879194630872, 0.19020134228187918)

In [22]:
b_group.sum_gamerounds.mean(), b_group.retention_1.mean(), b_group.retention_7.mean()

(51.29877552814966, 0.44228274967574577, 0.18200004396667327)

In [23]:
from statsmodels.stats.proportion import proportions_ztest

In [24]:
import numpy as np

In [30]:
info_a = np.array([a_group.retention_7.sum(), b_group.retention_7.sum()])

In [31]:
info_b = np.array([a_group.shape[0], b_group.shape[0]])

In [32]:
info_a, info_b

(array([8502, 8279]), array([44700, 45489]))

In [33]:
proportions_ztest(info_a, info_b)

(3.164358912748191, 0.001554249975614329)

In [36]:
import statsmodels.stats.power as smp
import math

In [38]:
h = 2 * math.asin(np.sqrt(a_group.retention_7.mean())) - 2 * math.asin(np.sqrt(b_group.retention_7.mean()))

In [39]:
smp.zt_ind_solve_power(h, a_group.shape[0], 0.05)

0.8830912109776261

### **Рекомендации:** Перемещение ворот с 30 уровня на 40 ухудшило метрику возврата игроков через 7 дней после установки игры. Откатываем эксперимент